Now we want to populate our data repository

In [1]:
import json
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import numpy as np
import simplesimdb as simplesim # this one is our own database manager
import subprocess # to capture errors from the simulations
import itertools

In [2]:
m = simplesim.Manager(directory='advection', executable='./execute.sh', filetype='nc')

In [3]:
init2x= {"shear":[0.0,6.283185307179586],
         "mms": [-1.0,1.0]
         }
init2bc= {"shear":["PER", "DIR"],
          "mms": ["DIR", "PER"]}

init2init =  {
    "shear": {"type": "shear", "rho": 0.20943951023931953, "delta": 0.2 },
    "mms":   {"type": "mms", "sigma": 0.2, "velocity": 1.0}
}

n2v={
     "shear": {
    1:[144,1e-2],
    3:[48, 5e-3],
    4:[36, 2.5e-3],
    5:[28, 2e-3],
    7:[20, 1e-3]
    },
    "mms": {
    1:[144,2e-2],
    3:[48, 2e-3],
    4:[36, 2e-3],
    5:[28, 1e-3],
    7:[20, 5e-4]
    }
}

endtime = {"shear": 10, "mms": 1}

maxout = {"shear": 40, "mms": 5}
reg2reg = {"none" : {"type" : "none"},
           "modal" : {"type" : "modal", "alpha" : 36, "order" : 8, "eta_c" : 0.5},
          "viscous": {"type" : "viscosity", "order" : 1,
                      "nu_perp" : 1e-3, "direction" : "forward"},
          "hyperviscous" : {"type": "viscosity", "order" : 2,
                            "nu_perp" : 5e-4, "direction" : "forward"}
          }

In [4]:
def generate_standard_input( n, init, multiplication, advection, regularization) :
    itstp=endtime[init]/maxout[init]/n2v[init][n][1]
    if not itstp.is_integer() :
        print("Warning: itstp is not an integer!")
    inputfile={
        "grid":
        {   
            "type": "Cartesian2d",
            "n"  : n,
            "Nx" : n2v[init][n][0],
            "Ny" : n2v[init][n][0],
            "x": init2x[init],
            "y": init2x[init],
            "bc" : init2bc[init]
        },
        "timestepper":
        {   
            "type": "FilteredExplicitMultistep",
            "tableau": "eBDF-3-3",
            "dt" : n2v[init][n][1]
        },
        "regularization":  reg2reg[regularization],
        "output":
        {   
            "type": "netcdf",
            "itstp"   : int(itstp),
            "maxout"  : int(maxout[init])
        },
        "advection":
        {   
            "multiplication": multiplication,
            "type": advection
        },
        "elliptic":
        {   
            "type" : "multigrid",
            "stages": 3,
            "eps_pol" : [1e-6,10.0,10.0],
            "direction" : "forward"
        },
        "init": init2init[init]
    }
    if regularization == "viscous" or regularization == "hyperviscous" :
        inputfile["timestepper"]["type"] = "ImExMultistep"
        inputfile["timestepper"]["tableau"] = "ImEx-BDF-3-3"
        inputfile["timestepper"]["eps_time"] = 1e-9
    return inputfile

In [5]:
# THE RED BUTTON
# m.delete_all()

In [8]:
v_n=[1,3,4,5]
v_init=["shear", "mms"]
v_multiplication=["pointwise", "projection"]
v_advection=["arakawa","centered", "upwind"]
v_regularization=["none", "modal"]

for t in itertools.product( v_n, v_init, v_multiplication, v_advection,
                           v_regularization) :
    inputfile = generate_standard_input( 
        n=t[0], init=t[1],multiplication=t[2],
        advection=t[3],regularization=t[4])      

    ncfile=''
    # Check for existing data
    #if not m.exists(inputfile):
    #    print(t)
    # uncomment the following to create data
    try:
        print(t)
        ncfile=m.create(inputfile)
    # the simulation may fail 
    except subprocess.CalledProcessError as e:
        print( e.stderr)
    print( ncfile)

(1, 'shear', 'pointwise', 'arakawa', 'none')
advection/d28090c043aa4565de8dd1fc64c10de462cdbd85.nc
(1, 'shear', 'pointwise', 'arakawa', 'modal')
Running simulation ... 
 ... Done
advection/78126910b9675c15e2b87086a13517aac91ec1de.nc
(1, 'shear', 'pointwise', 'centered', 'none')
advection/411d2cee49763ed03f66ce5c25749656b4f33cba.nc
(1, 'shear', 'pointwise', 'centered', 'modal')
Running simulation ... 
 ... Done
advection/e984545727e07fe59854de38c84397359ea78f03.nc
(1, 'shear', 'pointwise', 'upwind', 'none')
advection/7a02fff5a16faa6648ec2615eea3637bd6aa5115.nc
(1, 'shear', 'pointwise', 'upwind', 'modal')
Running simulation ... 
 ... Done
advection/fa99c0fa20a8055062cc8fe17611037eb4dd5a8a.nc
(1, 'shear', 'projection', 'arakawa', 'none')
advection/76caf6ff55d2c2c00650316e694033a13c54ca95.nc
(1, 'shear', 'projection', 'arakawa', 'modal')
Running simulation ... 
 ... Done
advection/3289f27654ab07a04a55a06f305d199ce9f2ba45.nc
(1, 'shear', 'projection', 'centered', 'none')
Running simulation 

 ... Done
advection/c8dcfae370ae579e428c542a6b67d708514ab8be.nc
(4, 'mms', 'pointwise', 'centered', 'none')
advection/d883715de820670f54afce56f70ea947bab90a6c.nc
(4, 'mms', 'pointwise', 'centered', 'modal')
Running simulation ... 
 ... Done
advection/99835022e9456b35445cad3780ec54dc515add8c.nc
(4, 'mms', 'pointwise', 'upwind', 'none')
advection/ad1c308602f45f0c80bcf79765e12db434167347.nc
(4, 'mms', 'pointwise', 'upwind', 'modal')
Running simulation ... 
 ... Done
advection/84da0ebc54b3c13a6436eb61baf930120241ee4c.nc
(4, 'mms', 'projection', 'arakawa', 'none')
advection/e98aad3d8b85d5113d6d9764dac7a4d229f277e3.nc
(4, 'mms', 'projection', 'arakawa', 'modal')
Running simulation ... 
 ... Done
advection/d7389f5bcab0298000677e8d35576db3743c5dbb.nc
(4, 'mms', 'projection', 'centered', 'none')
advection/2005ebb318048501202203de19a04dc4af70beb2.nc
(4, 'mms', 'projection', 'centered', 'modal')
Running simulation ... 
 ... Done
advection/c049e9486142d18560d96c31492c2d387f04e0fb.nc
(4, 'mms', 'pr

In [7]:
# change the eta_c parameter to 0 for comparison
v_n=[3,4,5]#,7]
v_init=["shear"]
v_multiplication=["pointwise","projection"]
v_advection=["arakawa","centered","upwind"]
v_regularization=[ "modal"]

for t in itertools.product( v_n, v_init, v_multiplication, v_advection,
                           v_regularization) :
    inputfile = generate_standard_input( 
        n=t[0], init=t[1],multiplication=t[2],
        advection=t[3],regularization=t[4] ) 
    inputfile["regularization"]["eta_c"] = float(0)

    ncfile=''
    try:
        print(t)
        ncfile=m.create(inputfile)
    # the simulation may fail 
    except subprocess.CalledProcessError as e:
        print( e.stderr)

(3, 'shear', 'pointwise', 'arakawa', 'modal')
(3, 'shear', 'pointwise', 'centered', 'modal')
(3, 'shear', 'pointwise', 'upwind', 'modal')
(3, 'shear', 'projection', 'arakawa', 'modal')
(3, 'shear', 'projection', 'centered', 'modal')
(3, 'shear', 'projection', 'upwind', 'modal')
(4, 'shear', 'pointwise', 'arakawa', 'modal')
(4, 'shear', 'pointwise', 'centered', 'modal')
(4, 'shear', 'pointwise', 'upwind', 'modal')
(4, 'shear', 'projection', 'arakawa', 'modal')
(4, 'shear', 'projection', 'centered', 'modal')
(4, 'shear', 'projection', 'upwind', 'modal')
(5, 'shear', 'pointwise', 'arakawa', 'modal')
(5, 'shear', 'pointwise', 'centered', 'modal')
(5, 'shear', 'pointwise', 'upwind', 'modal')
(5, 'shear', 'projection', 'arakawa', 'modal')
(5, 'shear', 'projection', 'centered', 'modal')
(5, 'shear', 'projection', 'upwind', 'modal')


In [13]:
# Generate one good simulation with double resolution
inputfile = generate_standard_input( 
        n=5, init="shear",multiplication="projection",
        advection="upwind",regularization="none" )
inputfile["grid"]["Nx"] *=2
inputfile["grid"]["Ny"] *=2
inputfile["timestepper"]["dt"]/=2.0
inputfile["output"]["itstp"]*=2

print( inputfile)
ncfile=''
try:
    ncfile=m.create(inputfile)
    #m.delete(inputfile)
except subprocess.CalledProcessError as e:
    print( e.stderr)

{'grid': {'type': 'Cartesian2d', 'n': 5, 'Nx': 56, 'Ny': 56, 'x': [0.0, 6.283185307179586], 'y': [0.0, 6.283185307179586], 'bc': ['PER', 'DIR']}, 'timestepper': {'type': 'FilteredExplicitMultistep', 'tableau': 'eBDF-3-3', 'dt': 0.001}, 'regularization': {'type': 'none'}, 'output': {'type': 'netcdf', 'itstp': 250, 'maxout': 40}, 'advection': {'multiplication': 'projection', 'type': 'upwind'}, 'elliptic': {'type': 'multigrid', 'stages': 3, 'eps_pol': [1e-06, 10.0, 10.0], 'direction': 'forward'}, 'init': {'type': 'shear', 'rho': 0.20943951023931953, 'delta': 0.2}}
Running simulation ... 
 ... Done
